In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from astropy.table import Table
import gravity_obs.p2_tools as p2t

In [6]:
table_path = '../../gvprep/tabs/star_k9-12mag_grp10-12mag_gaia_pairs_on.ipac'
tb = Table.read(table_path, format='ascii.ipac')

In [7]:
tb

name,ra_2000_hms,dec_2000_dms,ra_2000,dec_2000,parallax,pm,pmra,pmdec,radial_velocity,Grp,G,ra_2016,dec_2016,source_id,name_ft,ra_2000_hms_ft,dec_2000_dms_ft,ra_2000_ft,dec_2000_ft,H_ft,K_ft,parallax_ft,pm_ft,pmra_ft,pmdec_ft,radial_velocity_ft,Grp_ft,G_ft,ra_2016_ft,dec_2016_ft,source_id_ft,sobj_x,sobj_y,sep,pa
,,,,,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,,,,,,mag,mag,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,mas,mas,arcsec,deg
str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
Gaia DR3 2314519534641619072,00:00:48.419,-31:24:08.551,0.2017465628668493,-31.402375276095544,1.9224,76.271,40.103,-64.878,--,11.25193,12.474266,0.20195538217,-31.40266362203,2314519534641619072,Gaia DR3 2314519534642211968,00:00:48.458,-31:24:08.366,0.20191001276,-31.402323892,10.355999947,10.262000084,1.8048,74.96,38.581,-64.269,--,11.243553,12.437418,0.20211090932,-31.402609531,2314519534642211968,-462.95471795708687,-200.70267317723417,0.5045875879361418,246.56203839541735
Gaia DR3 2365173519937952768,00:19:05.830,-19:33:02.667,4.774291555243661,-19.550740854562502,7.3023,47.847,-8.481,-47.089,--,10.239418,11.448545,4.77425155779,-19.55095013961,2365173519937952768,Gaia DR3 2365173519937707392,00:19:05.828,-19:33:03.195,4.7742837938,-19.55088739,9.1660003662,9.0640001297,6.5395,49.094,-15.887,-46.452,--,10.238993,11.66296,4.7742088645,-19.551093845,2365173519937707392,217.491843123125,511.08998598663896,0.5554418741874949,23.05194091486316
Gaia DR3 4998055954911196288,00:21:39.581,-38:45:46.511,5.41491879640779,-38.76291972863743,4.9668,18.786,-7.013,-17.428,11.36,10.953303,12.122323,5.41487882581,-38.76299718829,4998055954911196288,Gaia DR3 4998055954912245760,00:21:39.543,-38:45:46.605,5.4147635206,-38.76294592,9.9139995575,9.8269996643,3.3745,13.872,-8.13,-11.24,--,10.947227,12.356506,5.4147171814,-38.762995875,4998055954912245760,464.7058196694789,-65.43626197121604,0.4692903186892395,98.01524795234509
Gaia DR3 5004971745611061120,00:26:50.606,-35:30:11.178,6.710858731441873,-35.50310492673278,4.6755,29.369,18.597,22.73,--,10.648828,12.040381,6.71096026331,-35.5030039052,5004971745611061120,Gaia DR3 5004971741316137984,00:26:50.584,-35:30:10.747,6.7107662388,-35.502985171,9.6160001755,9.501999855,4.6923,27.399,13.186,24.017,--,10.641261,11.84198,6.7108382241,-35.502878428,5004971741316137984,410.746802994976,-464.3444458224562,0.6199424977663314,138.50487717159072
Gaia DR3 2363592078619156480,00:27:48.476,-19:28:55.327,6.951982344057993,-19.482035378287936,3.2245,22.732,-9.4,-20.697,--,10.217233,11.499516,6.95193803122,-19.48212736709,2363592078619156480,Gaia DR3 2363592074324188416,00:27:48.487,-19:28:55.818,6.9520273757,-19.482171594,9.6420001984,9.6129999161,2.6422,23.204,-9.939,-20.968,--,10.214808,11.143388,6.9519805182,-19.482264784,2363592074324188416,-138.90782872737245,497.35953549904536,0.5163931568423222,344.39545269218786
Gaia DR3 2346579816238844800,00:38:01.288,-25:47:33.425,9.505365061588092,-25.792618131015033,9.145,108.553,16.04,-107.362,24.9,10.73508,12.183564,9.50544424001,-25.79309529636,2346579816238844800,Gaia DR3 2346579816240252160,00:38:01.301,-25:47:33.914,9.5054188996,-25.792753835,9.1890001297,9.0480003357,8.9883,102.035,16.694,-100.66,--,10.735076,12.302464,9.5055013044,-25.793201213,2346579816240252160,-191.38104981645046,315.5515171485597,0.3690521185464249,328.7633139318653
Gaia DR3 2341873150558360448,00:44:53.233,-29:53:55.989,11.221806217785431,-29.89888569650614,4.6826,10.339,-1.206,10.268,-8.52,10.321059,11.573556,11.22180003449,-29.89884005906,2341873150558360448,Gaia DR3 2341873150557503488,00:44:53.216,-29:53:55.547,11.221733346,-29.898763104,9.373000145,9.343000412,5.1521,6.674,-3.192,5.862,--,10.32097,11.495355,11.221716982,-29.898737052,2341873150557503488,

In [9]:
tb.add_index('name')
tb['K'] = tb['G']-2.5
tb['H'] = tb['G']-2.2

In [10]:
from gravity_obs import ob_add_target
def dual_on_for_target(grav, target, folder_name=None,
                         usename=False, verify=False):
    ao_mode = "LGS_VIS"
    tel_alpha = target['ra_2000_hms']
    tel_delta = target['dec_2000_dms']

    if usename:
            obname = f"{ao_mode}_{target['name']}".replace(' ', '_')
    else:
            obname = f"{ao_mode}_{tel_alpha.split('.')[0].replace(':', '')}{tel_delta.split('.')[0].replace(':', '')}_{target['Grp']:4.1f}_{target['K']:4.1f}".replace(' ', '_')


    ob, obV = grav.create_OB(obname, folder_name=folder_name, overwrite=True)
    obsDes = ob['obsDescription']
    obsDes['name'] = target['name']
    obsDes['userComments'] = 'Dual on-axis observation test'

    ob_add_target(ob, target['name'], 
                    ra = target['ra_2000_hms'], 
                    dec = target['dec_2000_dms'],
                    pma = target['pmra'],
                    pmd = target['pmdec'])
    ob, obV = grav.save_OB(obname, folder_name=folder_name)


    dit, ndit = 3, 40
    grav.add_GRAVITY_dual_onaxis_acq(name=obname, folder_name=folder_name,
                                    sc_name=target['name'],
                                    sc_kmag=target['K'],
                                    sc_hmag=target['H'],
                                    sobj_x = target['sobj_x'],
                                    sobj_y = target['sobj_y'],
                                    sc_plx=target['parallax']/1e3,
                                    ft_kmag=target['K_ft'],
                                    ft_hmag=target['H_ft'],
                                    ft_plx=target['parallax_ft']/1e3,
                                    ao_mode=ao_mode,
                                    ngs_mag=target['Grp_ft'],
                                    baseline=['UTs'])
    grav.add_GRAVITY_dual_obs_exp(obname, folder_name=folder_name,
                                    dit=dit, ndit_obj=ndit, ndit_sky=ndit,)

    # optional verification
    if verify:
        obID = ob['obId']
        response, _ = grav.api.verifyOB(obID,True)
        if response['observable']:
            print('*** Congratulations. Your OB' , obID, ob['name'], 'is observable!')
        else:
            print('OB', obID, 'is >>not observable<<. See messages below.')
            print(' ', '\n  '.join(response['messages']))

In [11]:
grav = p2t.p2api_GRAVITY('116.29H7.001', 'GPLUSTeam', 'GPAOwFRINGES', no_warning=True)
root_folder = 'TTR-116.0046 October First Light'
grav.set_rootFolder(root_folder)

In [12]:
folder_name = 'dual_onaxis'
try:
    grav.get_folder(folder_name)
except Exception:
    grav.create_folder(folder_name, overwrite=True)

In [13]:
target_names = ["Gaia DR3 6619188440209039744", "Gaia DR3 5003111440656257408"]

for obj_name in target_names:
    target = tb.loc[obj_name]
    dual_on_for_target(grav, target, folder_name=folder_name)